In [1]:
import geopandas as gpd

## Open GIS Data

In [2]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_gdf = gpd.read_file(file_path)
states_gdf = states_gdf[["STUSPS", "geometry"]]

## Open Chipotle Data

In [3]:
chipotle_gdf = gpd.read_file("data/Chipotle_Per_State.gpkg")
chipotle_gdf["chipotle_per_100k"] = chipotle_gdf["per_100k"]
chipotle_gdf = chipotle_gdf[["STUSPS", "Chipotles", "chipotle_per_100k"]]

## Open QDOBA Data

In [4]:
qdoba_gdf = gpd.read_file("data/QDOBA_Per_State.gpkg")
qdoba_gdf["QDOBA_per_100k"] = qdoba_gdf["per_100k"]
qdoba_gdf = qdoba_gdf[["STUSPS", "QDOBAs", "QDOBA_per_100k"]]

## Merge Data

In [5]:
chipotle_states_gdf = states_gdf.merge(chipotle_gdf, on="STUSPS", how="left")
chipotle_qdoba_states_gdf = chipotle_states_gdf.merge(
    qdoba_gdf, on="STUSPS", how="left"
)

In [6]:
chipotle_qdoba_states_gdf = chipotle_qdoba_states_gdf.fillna(0)

In [7]:
chipotle_qdoba_states_gdf["QDOBAs"] = chipotle_qdoba_states_gdf["QDOBAs"].astype(int)
chipotle_qdoba_states_gdf["Chipotles"] = chipotle_qdoba_states_gdf["Chipotles"].astype(
    int
)

In [8]:
def more_stores(row) -> str:
    qdoba = row["QDOBAs"]
    chipotle = row["Chipotles"]
    if qdoba > chipotle:
        return "More QDOBAs"
    if qdoba < chipotle:
        return "More Chipotles"
    if qdoba == 0 and chipotle == 0:
        return "No Locations"
    if qdoba == chipotle:
        return "Same Amount of Chipotles & QDOBAs"
    else:
        return "Edge Case"

In [9]:
chipotle_qdoba_states_gdf["label"] = chipotle_qdoba_states_gdf.apply(
    lambda row: more_stores(row), axis=1
)

In [10]:
chipotle_qdoba_states_gdf[chipotle_qdoba_states_gdf["label"] == "Edge Case"]

,STUSPS,geometry,Chipotles,chipotle_per_100k,QDOBAs,QDOBA_per_100k,label


In [11]:
chipotle_qdoba_states_gdf = chipotle_qdoba_states_gdf.to_crs(9311)
chipotle_qdoba_states_gdf.to_file("data/Chipotle_Vs_QDOBA_Per_State.gpkg")